In [20]:
import cv2
import numpy as np
import os
import requests

# URLs for YOLOv3 files
yolo_weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
yolo_cfg_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg'
coco_names_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'

# Filenames for the YOLOv3 files
yolo_weights_file = 'yolov3.weights'
yolo_cfg_file = 'yolov3.cfg'
coco_names_file = 'coco.names'

# Function to download a file if it doesn't exist
def download_file(url, filename):
    if not os.path.isfile(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"{filename} downloaded.")
    else:
        print(f"{filename} already exists.")

# Download the YOLOv3 files
download_file(yolo_weights_url, yolo_weights_file)
download_file(yolo_cfg_url, yolo_cfg_file)
download_file(coco_names_url, coco_names_file)


yolov3.weights already exists.
yolov3.cfg already exists.
coco.names already exists.


In [21]:
# Load YOLOv3 model
net = cv2.dnn.readNet(yolo_weights_file, yolo_cfg_file)

# Load COCO class labels
with open(coco_names_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Get the output layer names from the YOLO model
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Assign random colors to each class label
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [22]:
# Load the input image
image = cv2.imread('sample_input.webp')
height, width, channels = image.shape

# Prepare the image for YOLO
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Process the detections
class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Maximum Suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw the results on the image
for i in indices:
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    color = colors[class_ids[i]]
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

In [23]:
output_image = 'output_image.jpg'
cv2.imwrite(output_image, image)


True